##### - 가천대학교 기계학습프로그래밍 과목 기말 팀 프로젝트 과제
##### - 팀명: 띵킹
##### - 주제: 텍스트 기반 사용자의 감정을 분석하여 감정에 따른 음악 추천 서비스
##### - 코드 작성일: 2023-11-16

#1. 한국어_단발성_대화_데이터셋 KoBERT 모델 시도 결과

In [ ]:
# 필요한 라이브러리 설치
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers
!pip install torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 18.7 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.1
    Uninstalling graphviz-0.20.1:
      Successfully uninstalled graphviz-0.20.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 344.5/344.5 kB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp310-cp310-linux_x86_64.whl size=661675 sha256=e8588d6d333e9691fb774aa4a7d900c048c3191fa63d516b195e92fe9b547d06
  Stored in directory: /root/.cache/pip/wheels/1a/1e/0d/99f55911d90f2b95b9f7c176d5813ef3622894a4b30fde6bd3
Successfully built gluonnlp
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.7/311.7 kB 41.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 54.7 MB/s eta 0:00:00
     ━━━━━━

In [ ]:
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-fv9rdj11/kobert-tokenizer_d764c4f77b074de192733d1d3030d40f
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-fv9rdj11/kobert-tokenizer_d764c4f77b074de192733d1d3030d40f
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
  Created wheel for kobert_tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4632 sha256=ef9f4865a444f6082c1fdf40ca813cbae67a88d09d3248b0a5264d55b0ee1bb4
  Stored in directory: /tmp/pip-ephem-wheel-cache-m8bxijvv/wheels/e9/1a/3f/a864970e8a169c176befa3c4a1e07aa612f69195907a4045fe
Successfully built kobert_tokenizer


In [ ]:
# 라이브러리 임포트
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [ ]:
# Hugging Face를 통한 모델 및 토크나이저 Import
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
# GPU 사용 시
device = torch.device("cuda:0")

In [ ]:
# 문장 토큰화 클래스

class BERTSentenceTransform:
    r"""BERT 스타일의 데이터 변환 클래스

    Parameters
    ----------
    tokenizer : BERTTokenizer.
        문장에 대한 토크나이저
    max_seq_length : int.
        문장의 최대 시퀀스 길이
    pad : bool, default True
        문장을 최대 길이에 맞게 패딩할지 여부
    pair : bool, default True
        문장 또는 문장 쌍을 변환할지 여부
    """

    def __init__(self, tokenizer, max_seq_length,vocab, pad=True, pair=True):
        self._tokenizer = tokenizer
        self._max_seq_length = max_seq_length
        self._pad = pad
        self._pair = pair
        self._vocab = vocab

    def __call__(self, line):
        """문장 쌍 또는 단일 문장에 대한 변환 수행.

        변환은 다음 단계로 처리:
        - 입력 문장을 토큰화.
        - 필요한 경우 [CLS], [SEP]를 삽입.
        - 토큰이 첫 번째 시퀀스에 속하는지 두 번째 시퀀스에 속하는지 나타내는 타입 ID를 생성.
        - 유효한 길이를 생성.


        시퀀스 쌍의 경우, 입력은 2개의 문자열로 구성된 튜플:

            text_a: 'is this jacksonville ?'
            text_b: 'no it is not'
        Tokenization: 각 문장은 BERT 토크나이저를 사용하여 토큰화됩니다.
            text_a: 'is this jack ##son ##ville ?'
            text_b: 'no it is not .'
        Processed: 최종 처리된 결과는 다음과 같습니다.
            tokens: '[CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]' - [CLS]는 문장의 시작, [SEP]는 문장의 종료를 나타냄.
            type_ids: 0 0 0 0 0 0 0 0 1 1 1 1 1 1 - 첫 번째 문장의 토큰에는 0이, 두 번째 문장의 토큰에는 1이 할당.
        valid_length: 14 - 실제로 사용된 토큰의 길이


        단일 시퀀스의 경우, 입력은 단일 문자열로 구성된 튜플.

        입력:
            text_a: 'the dog is hairy .'
        토큰화:
            text_a: 'the dog is hairy .'
        처리된 결과:
            text_a: '[CLS] the dog is hairy . [SEP]'
            type_ids: 0     0   0   0  0     0 0
            valid_length: 7



        **반환값**
          -------
          np.array: 'int32' 형식의 입력 토큰 ID, 형태 (배치 크기, 시퀀스 길이)
          np.array: 'int32' 형식의 유효한 길이, 형태 (배치 크기,)
          np.array: 'int32' 형식의 입력 토큰 타입 ID, 형태 (배치 크기, 시퀀스 길이)

        """

        # 유니코드로 변환
        text_a = line[0]
        if self._pair:
            assert len(line) == 2
            text_b = line[1]

        tokens_a = self._tokenizer.tokenize(text_a)
        tokens_b = None

        if self._pair:
            tokens_b = self._tokenizer(text_b)

        if tokens_b:
            # 지정된 길이보다 작도록 `tokens_a` 및 `tokens_b`를 수정
            self._truncate_seq_pair(tokens_a, tokens_b,
                                    self._max_seq_length - 3)
        else:
            if len(tokens_a) > self._max_seq_length - 2:
                tokens_a = tokens_a[0:(self._max_seq_length - 2)]

        vocab = self._vocab
        tokens = []
        tokens.append(vocab.cls_token)
        tokens.extend(tokens_a)
        tokens.append(vocab.sep_token)
        segment_ids = [0] * len(tokens)

        if tokens_b:
            tokens.extend(tokens_b)
            tokens.append(vocab.sep_token)
            segment_ids.extend([1] * (len(tokens) - len(segment_ids)))

        input_ids = self._tokenizer.convert_tokens_to_ids(tokens)

        # 문장의 유효 길이. 실제 토큰만 참석
        valid_length = len(input_ids)

        if self._pad:
             # 시퀀스 길이까지 0으로 채움.
            padding_length = self._max_seq_length - valid_length
            # 나머지 부분에 대해 패딩 토큰을 사용.
            input_ids.extend([vocab[vocab.padding_token]] * padding_length)
            segment_ids.extend([0] * padding_length)

        return np.array(input_ids, dtype='int32'), np.array(valid_length, dtype='int32'),\
            np.array(segment_ids, dtype='int32')


In [ ]:
import pandas as pd

# 엑셀 파일 불러오기
df = pd.read_excel('/content/한국어_단발성_대화_데이터셋.xlsx')

# 필요없는 열 삭제
df = df[['Sentence', 'Emotion']]

In [ ]:
# 'Emotion' 열에서 '혐오'를 '분노'로 변경 (한국어_단발성_대화_데이터셋)
df['Emotion'] = df['Emotion'].replace('혐오', '분노')

# 변경된 데이터프레임 확인
df[df['Emotion'] == '분노']

# 'Emotion' 열에서 '중립'을 '평온'으로 변경 (한국어_단발성_대화_데이터셋)
df['Emotion'] = df['Emotion'].replace('중립', '평온')

# 변경된 데이터프레임 확인
df[df['Emotion'] == '평온']

# 'Emotion' 열에서 '공포'를 '불안'으로 변경 (한국어_단발성_대화_데이터셋)
df['Emotion'] = df['Emotion'].replace('공포', '불안')

# 'Emotion' 열에서 '놀람'을 '당황'으로 변경 (한국어_단발성_대화_데이터셋)
df['Emotion'] = df['Emotion'].replace('놀람', '당황')

# 'Emotion' 열에서 '행복'을 '기쁨'으로 변경 (한국어_단발성_대화_데이터셋)
df['Emotion'] = df['Emotion'].replace('행복', '기쁨')

In [ ]:
# 감정 카테고리 숫자 데이터로 변환

df.loc[(df['Emotion'] == "불안"), 'Emotion'] = 0  #공포 => 0
df.loc[(df['Emotion'] == "당황"), 'Emotion'] = 1  #놀람 => 1
df.loc[(df['Emotion'] == "분노"), 'Emotion'] = 2  #분노 => 2
df.loc[(df['Emotion'] == "슬픔"), 'Emotion'] = 3  #슬픔 => 3
df.loc[(df['Emotion'] == "평온"), 'Emotion'] = 4  #평온 => 4
df.loc[(df['Emotion'] == "기쁨"), 'Emotion'] = 5  #행복 => 5

In [ ]:
# 학습용, 테스트용 데이터분리
from sklearn.model_selection import train_test_split

X = df[['Sentence']]
y = df['Emotion']

# 데이터 분리, 20%의 데이터를 테스트용으로 사용
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 나눠진 데이터 개수 확인
print("학습 데이터 개수:", len(X_train))
print("테스트 데이터 개수:", len(X_test))

학습 데이터 개수: 30875
테스트 데이터 개수: 7719


In [ ]:
# 학습용, 테스트용 데이터 생성
dataset_train = pd.concat([X_train, y_train], axis=1)
dataset_test = pd.concat([X_test, y_test], axis=1)

In [ ]:
from gluonnlp.data import TSVDataset

# tsv 데이터로 변환
# sep='\t'탭을 구분자로 사용하여 csv가 아닌 tsv파일로 변환한다
dataset_train.to_csv('dataset_train.tsv', sep='\t', index=False)
dataset_test.to_csv('dataset_test.tsv', sep='\t', index=False)

In [ ]:
# TSV 파일에서 데이터셋을 로드
# num_discard_samples=1 첫번째 행을 무시 -> 열 이름

dataset_train = nlp.data.TSVDataset('dataset_train.tsv', field_indices=[0, 1], num_discard_samples=1)
dataset_test = nlp.data.TSVDataset('dataset_test.tsv', field_indices=[0, 1], num_discard_samples=1)

In [ ]:
# 파라미터 세팅
max_len = 64            # 문장 최대 길이
batch_size = 64         # 한 번의 모델 업데이트를 위해 사용되는 데이터 샘플의 개수
warmup_ratio = 0.1      # 학습 초기에 적용되는 웜업 비율
num_epochs = 1          # 전체학습 반복 횟수
max_grad_norm = 1       # 그래디언트 폭주를 방지
log_interval = 200      # 학습 중 로그를 출력하는 간격
learning_rate =  5e-5   # 학습률

In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len,
                 pad, pair):

        # BERTSentenceTransform을 사용하여 데이터셋의 문장과 레이블을 변환
        transform = BERTSentenceTransform(bert_tokenizer, max_seq_length=max_len,vocab=vocab, pad=pad, pair=pair)
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        # 변환된 문장과 레이블을 반환
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

# KoBERT 토크나이저 및 BERT 모델 불러오기
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)

# BERTVocab을 사용하여 vocab 생성
# (vocab => 모델이 텍스트 데이터를 처리하는 과정에서 사용되는 어휘 집합)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')

# 학습 데이터셋과 테스트 데이터셋을 BERTDataset으로 변환
data_train = BERTDataset(dataset_train, 0, 1, tokenizer, vocab, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tokenizer, vocab, max_len, True, False)

(…)se-v1/resolve/main/tokenizer_config.json:   0%|          | 0.00/432 [00:00<?, ?B/s]

(…)kobert-base-v1/resolve/main/spiece.model:   0%|          | 0.00/371k [00:00<?, ?B/s]

(…)-v1/resolve/main/special_tokens_map.json:   0%|          | 0.00/244 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


(…)/kobert-base-v1/resolve/main/config.json:   0%|          | 0.00/535 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/369M [00:00<?, ?B/s]

In [ ]:
# 데이터셋을 미니배치로 나누어 준다, 현재 설정된 batch_size = 64
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


================================================================================


In [ ]:
# 감정 분류를 위한 분류 모델을 정의하는 클래스
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=6, # 클래스 수, 감정 분류 개수에 따라 바뀜
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
# BERTClassifier 인스턴스 생성
# - bertmodel: 미리 학습된 BERT 모델
# - dr_rate: 드롭아웃 비율 (옵션, 0.5로 설정)
# 드롭 아웃 -> 드롭아웃은 모델이 학습할 때 무작위로 일부 뉴런을 비활성화하여
#              모델의 일반화 성능을 향상시키는 데 사용.
#              여기서는 0.5로 설정되어 있어 각 뉴런이 50%의 확률로 비활성화.
# .to(device): 모델을 GPU에서 사용

model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [ ]:
# Optimizer와 학습 스케줄러 설정 (linear warmup and decay)
# 모델의 일반화를 향상시키고 오버피팅을 방지하는 데 도움

# 가중치 감쇠를 적용하지 않을 파라미터 그룹을 정의
no_decay = ['bias', 'LayerNorm.weight']

# 가중치 감쇠를 적용하지 않는 파라미터 그룹은 weight_decay를 0.0으로 설정하고,
# 적용하는 파라미터 그룹은 weight_decay를 0.01로 설정
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [ ]:
# Optimizer 설정: AdamW를 사용하며, 설정된 파라미터 그룹과 학습률을 적용
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)

# 손실 함수 설정: CrossEntropyLoss를 사용
loss_fn = nn.CrossEntropyLoss()

# 전체 학습 스텝 수 및 웜업 스텝 수 계산
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

# 스케줄러 설정
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
# 정확도 계산 함수
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)

    # 정확도 계산: 예측값과 실제 레이블이 일치하는 경우를 계산
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
import pandas as pd
import random
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0

    # 모델을 학습 모드로 설정
    model.train()

    # 모델 학습
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)

        # 옵티마이저 업데이트 및 학습률 스케줄러 업데이트
        optimizer.step()
        scheduler.step()

        # 정확도 누적
        train_acc += calc_accuracy(out, label)

        # 일정 간격으로 학습 결과 출력
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))

    # 전체 데이터에 대해 한번의 학습이 끝난 후(에폭이 끝난 후) 학습 정확도 출력
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))

    # 모델을 평가 모드로 설정
    model.eval()

    # 테스트 데이터에 대한 정확도 계산
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)

        # 모델의 정확도 계산
        test_acc += calc_accuracy(out, label)

    # 에폭 종료 후 테스트 정확도 출력
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

<ipython-input-22-c711125859bb>:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/483 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


epoch 1 batch id 1 loss 1.85934579372406 train acc 0.125
epoch 1 batch id 201 loss 0.9523141384124756 train acc 0.4587997512437811
epoch 1 batch id 401 loss 1.0241352319717407 train acc 0.5316006857855362
epoch 1 train acc 0.5461033950617284


<ipython-input-22-c711125859bb>:38: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):


  0%|          | 0/121 [00:00<?, ?it/s]

epoch 1 test acc 0.6241093187115915


In [23]:
model1 = model # 원본 모델 복사

#감정기반음악추천

In [29]:
import pandas as pd
import random

# csv 파일 읽기
df = pd.read_csv('song_list.csv')

# 데이터 파일 경로
happy_music = df.loc[df['category'] == 0]   # 신나는 음악 = list[0]
calm_music = df.loc[df['category'] == 1]    # 잔잔한 음악 = list[1]
sad_music = df.loc[df['category'] == 2]     # 위로하는 음악 = list[2]
love_music = df.loc[df['category'] == 3]    # 달달한 음악 = list[3]
change_music = df.loc[df['category'] == 4]  # 기분전환음악 = list[4]


recommended_songs = []

# 모델을 통한 예측
def predict(predict_sentence):

  # 입력 문장과 임의의 레이블 데이터 생성
  data = [predict_sentence, '0']
  dataset_another = [data]

  # 입력 문장도 똑같이 토큰화
  another_test = BERTDataset(dataset_another, 0, 1, tokenizer, vocab, max_len, True, False)
  test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)

  # 모델을 평가 모드로 설정
  model1.eval()

  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
          token_ids = token_ids.long().to(device)
          segment_ids = segment_ids.long().to(device)
          valid_length= valid_length
          label = label.long().to(device)

          # 모델을 통해 예측 수행
          out = model1(token_ids, valid_length, segment_ids)

          for i in out:

            # 소프트맥스 함수를 적용하여 확률값 계산
            probs = torch.nn.functional.softmax(out, dim=-1)
            probs = probs.detach().cpu().numpy()

            # 가장 높은 확률을 갖는 클래스 선택
            predicted_class = np.argmax(probs, axis=1)[0]

            # 확률 계산
            probability = probs[0, predicted_class]*100

            # 결과에 따라 출력할 감정 카테고리
            class_names = ["불안", "당황", "분노", "슬픔", "평온", "행복",]

            recommended_songs = []

            # 감정에 따라 다른 데이터 로드
            if class_names[predicted_class] == '불안':
                random_number = random.choice([0, 1])

                if(random_number == 0): # 잔잔한 음악와 위로가 되는 음악 중 하나를 랜덤으로 선택
                  recommendation = ("마음 안정을 위한 잔잔한 분위기의 음악")# <분위기:장르>
                  recommended_songs = calm_music.sample(n=3) #랜덤음악 3개 추출

                elif(random_number == 1):
                  recommendation = ("힘내 한마디, 위로 되는 음악")
                  recommended_songs = sad_music.sample(n=3) #랜덤음악 3개 추출

            if class_names[predicted_class] == '당황':
                random_number = random.choice([0, 1])

                if(random_number == 0): # 잔잔한 음악와 기분전환이 되는 음악 중 하나를 랜덤으로 선택
                  recommendation = ("마음 안정을 위한 잔잔한 분위기의 음악")# <분위기:장르>
                  recommended_songs = calm_music.sample(n=3) #랜덤음악 3개 추출

                elif(random_number == 1):
                  recommendation = ("산뜻한 시작을 위한 기분전환이 되는 음악")
                  recommended_songs = change_music.sample(n=3) #랜덤음악 3개 추출

            if class_names[predicted_class] == '분노':
                random_number = random.choice([0, 1])

                if(random_number == 0): # 잔잔한 음악과 기분전환이 되는 음악 중 하나를 랜덤으로 선택
                  recommendation = ("마음 안정을 위한 잔잔한 분위기의 음악")# <분위기:장르>
                  recommended_songs = calm_music.sample(n=3)#랜덤음악 3개 추출

                elif(random_number == 1):
                  recommendation = ("산뜻한 시작을 위한 기분전환이 되는 음악")
                  recommended_songs = change_music.sample(n=3)#랜덤음악 3개 추출

            if class_names[predicted_class] == '슬픔':
                random_number = random.choice([0, 1])
                if(random_number == 0): #  신나는 음악와 위로가 되는 음악 중 하나를 랜덤으로 선택
                  recommendation = ("텐션 UP 신나는 음악")# <분위기:장르>
                  recommended_songs = happy_music.sample(n=3)#랜덤음악 3개 추출

                elif(random_number == 1):
                  recommendation = ("힘내 한마디. 위로가 되는 음악")
                  recommended_songs = sad_music.sample(n=3)#랜덤음악 3개 추출

            if class_names[predicted_class] == '평온':
                random_number = random.choice([0, 1])
                if(random_number == 0): # 달달한 음악과 기분전환이 되는 음악 중 하나를 랜덤으로 선택
                  recommendation = ("두근거리는 설렘을 위한 달달한 분위기의 음악")# <분위기:장르>
                  recommended_songs = love_music.sample(n=3)#랜덤음악 3개 추출

                elif(random_number == 1):
                  recommendation = ("산뜻한 시작을 위한 기분전환이 되는 음악")
                  recommended_songs = change_music.sample(n=3)#랜덤음악 3개 추출

            if class_names[predicted_class] == '행복':
                random_number = random.choice([0, 1])
                if(random_number == 0): # 신나는 음악과 달달한 음악 중 하나를 랜덤으로 선택
                  recommendation = ("두근거리는 설렘을 위한 달달한 분위기의 음악")# <분위기:장르>
                  recommended_songs = love_music.sample(n=3)#랜덤음악 3개 추출

                elif(random_number == 1):
                  recommendation = ("텐션 UP 신나는 음악")
                  recommended_songs = happy_music.sample(n=3)#랜덤음악 3개 추출




            # 추천 음악 데이터 출력
            print(f"당신의 감정은 {probability:.2f}% 확률로 {class_names[predicted_class]}으로 예측됩니다!" )
            print(f"저희가 {recommendation}을 추천해드릴게요.")

            print()

            print("추천 음악 :")
            print(f'{recommended_songs.iloc[0, 0]} - {recommended_songs.iloc[0, 1]}')
            print(f'{recommended_songs.iloc[1, 0]} - {recommended_songs.iloc[1, 1]}')
            print(f'{recommended_songs.iloc[2, 0]} - {recommended_songs.iloc[2, 1]}')


In [30]:
# 질문 무한 반복, 1을 입력하면 종료
while True:
  sentence = input("하고싶은 말을 입력해주세요 : ")
  if sentence == "1":
    break
  print(sentence)
  predict(sentence)
  print("\n")


하고싶은 말을 입력해주세요 : 오늘은 눈이와
오늘은 눈이와


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
<ipython-input-29-9e10eee97abc>:31: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):


  0%|          | 0/1 [00:00<?, ?it/s]

당신의 감정은 47.97% 확률로 당황으로 예측됩니다!
저희가 마음 안정을 위한 잔잔한 분위기의 음악을 추천해드릴게요.

추천 음악 :
비누인형 - 카니발
Rain is fallin' - 디어 (d.ear)
그네 - 달수빈


하고싶은 말을 입력해주세요 : 집에 가고싶어
집에 가고싶어


  0%|          | 0/1 [00:00<?, ?it/s]

당신의 감정은 71.73% 확률로 슬픔으로 예측됩니다!
저희가 텐션 UP 신나는 음악을 추천해드릴게요.

추천 음악 :
Stuck In The Middle - MIKA
HAPPY TOGETHER - 박지헌&강민경
Designer Drug (Bonus Track) - Mayer Hawthorne


하고싶은 말을 입력해주세요 : 오늘 점심에 짜장면을 먹었어
오늘 점심에 짜장면을 먹었어


  0%|          | 0/1 [00:00<?, ?it/s]

당신의 감정은 44.24% 확률로 행복으로 예측됩니다!
저희가 두근거리는 설렘을 위한 달달한 분위기의 음악을 추천해드릴게요.

추천 음악 :
Les Petits Miroirs (The Small Mirrors / 작은 거울들) - peppermoon
너는 나에게 - PPCX (피터팬컴플렉스)
Impressions of You - Kelly Finnigan


하고싶은 말을 입력해주세요 : 1


#===============================================

#2. 감성대화말뭉치 데이터셋 KoBERT 모델 시도 결과

In [ ]:
import pandas as pd

# 엑셀 파일 불러오기
df1 = pd.read_excel('/content/감성대화말뭉치(최종데이터)_Training.xlsx')
df2 = pd.read_excel('/content/감성대화말뭉치(최종데이터)_Validation.xlsx')

In [ ]:
# 필요없는 열 삭제
df1 = df1[['사람문장1', '감정_대분류']]
df2 = df2[['사람문장1', '감정_대분류']]

# 열 이름 변경
df1.rename(columns={'사람문장1': 'Sentence', '감정_대분류': 'Emotion'}, inplace=True)
df2.rename(columns={'사람문장1': 'Sentence', '감정_대분류': 'Emotion'}, inplace=True)

# 데이터프레임 합치기 (감성대화말뭉치)
combined_df = pd.concat([df1, df2], ignore_index=True)

<ipython-input-96-34958a9a57fd>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1.rename(columns={'사람문장1': 'Sentence', '감정_대분류': 'Emotion'}, inplace=True)
<ipython-input-96-34958a9a57fd>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2.rename(columns={'사람문장1': 'Sentence', '감정_대분류': 'Emotion'}, inplace=True)


In [ ]:
# 'Emotion' 열에서 '상처'를 '슬픔'으로 변경 (감성대화말뭉치)
combined_df['Emotion'] = combined_df['Emotion'].replace('상처', '슬픔')

# 줄바꿈 문자를 공백으로 대체 ('TSVDataset' 로드할 때 오류가 발생해서)
combined_df['Sentence'] = combined_df['Sentence'].str.replace('\n', ' ', regex=True)

# 변경된 데이터프레임 확인
combined_df[combined_df['Emotion'] == '슬픔']

,Sentence,Emotion
18,코로나 때문에 뭘 할 수가 없어. 취직 준비를 해야 하는데 시험이 줄줄이 취소되니 ...,슬픔
19,오늘 회사에서 큰 실수를 한 것 같아.,슬픔
20,요즘 취업 관련해서 떠올리기만 해도 온몸이 마비될 것 같아.,슬픔
21,어제도 야근 오늘도 야근이야. 너무 힘들어.,슬픔
25,면접관에게 완전히 속았어. 면접일에 알려준 연봉과 실수령액이 꽤 차이가 나네.,슬픔
...,...,...
58250,오늘 친구가 나랑 똑같은 치마를 입고 온 거 있지? 얼마나 기분 나빴는지 정말!,슬픔
58262,나 너무 억울하고 속상한 일이 있어.,슬픔
58263,친구 결혼식에 갔었는데 내가 안 온 줄 알고 친구가 토라져서 억울해!,슬픔
58264,나는 결혼을 안 하는 건데 주변에서는 결혼을 못 하는 거로 생각해.,슬픔


In [ ]:
# 감정 카테고리 숫자 데이터로 변환

combined_df.loc[(combined_df['Emotion'] == "불안"), 'Emotion'] = 0  #공포 => 0
combined_df.loc[(combined_df['Emotion'] == "당황"), 'Emotion'] = 1  #놀람 => 1
combined_df.loc[(combined_df['Emotion'] == "분노"), 'Emotion'] = 2  #분노 => 2
combined_df.loc[(combined_df['Emotion'] == "슬픔"), 'Emotion'] = 3  #슬픔 => 3
combined_df.loc[(combined_df['Emotion'] == "평온"), 'Emotion'] = 4  #평온 => 4
combined_df.loc[(combined_df['Emotion'] == "기쁨"), 'Emotion'] = 5  #행복 => 5

In [ ]:
# 학습용, 테스트용 데이터분리
from sklearn.model_selection import train_test_split

X = combined_df[['Sentence']]
y = combined_df['Emotion']

# 데이터 분리, 20%의 데이터를 테스트용으로 사용
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 나눠진 데이터 개수 확인
print("학습 데이터 개수:", len(X_train))
print("테스트 데이터 개수:", len(X_test))

학습 데이터 개수: 46616
테스트 데이터 개수: 11655


In [ ]:
# 학습용, 테스트용 데이터 생성
dataset_train = pd.concat([X_train, y_train], axis=1)
dataset_test = pd.concat([X_test, y_test], axis=1)

In [ ]:
from gluonnlp.data import TSVDataset

# tsv 데이터로 변환
# sep='\t'탭을 구분자로 사용하여 csv가 아닌 tsv파일로 변환한다
dataset_train.to_csv('dataset_train.tsv', sep='\t', index=False)
dataset_test.to_csv('dataset_test.tsv', sep='\t', index=False)

In [ ]:
# TSV 파일에서 데이터셋을 로드
# num_discard_samples=1 첫번째 행을 무시 -> 열 이름

dataset_train = nlp.data.TSVDataset('dataset_train.tsv', field_indices=[0, 1], num_discard_samples=1)
dataset_test = nlp.data.TSVDataset('dataset_test.tsv', field_indices=[0, 1], num_discard_samples=1)

In [ ]:
# 파라미터 세팅
max_len = 64            # 문장 최대 길이
batch_size = 64         # 한 번의 모델 업데이트를 위해 사용되는 데이터 샘플의 개수
warmup_ratio = 0.1      # 학습 초기에 적용되는 웜업 비율
num_epochs = 1          # 전체학습 반복 횟수
max_grad_norm = 1       # 그래디언트 폭주를 방지
log_interval = 200      # 학습 중 로그를 출력하는 간격
learning_rate =  5e-5   # 학습률

In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len,
                 pad, pair):

        # BERTSentenceTransform을 사용하여 데이터셋의 문장과 레이블을 변환
        transform = BERTSentenceTransform(bert_tokenizer, max_seq_length=max_len,vocab=vocab, pad=pad, pair=pair)
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        # 변환된 문장과 레이블을 반환
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

# KoBERT 토크나이저 및 BERT 모델 불러오기
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)

# BERTVocab을 사용하여 vocab 생성
# (vocab => 모델이 텍스트 데이터를 처리하는 과정에서 사용되는 어휘 집합)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')

# 학습 데이터셋과 테스트 데이터셋을 BERTDataset으로 변환
data_train = BERTDataset(dataset_train, 0, 1, tokenizer, vocab, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tokenizer, vocab, max_len, True, False)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [ ]:
# 데이터셋을 미니배치로 나누어 준다, 현재 설정된 batch_size = 64
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
# 감정 분류를 위한 분류 모델을 정의하는 클래스
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=6, # 클래스 수, 감정 분류 개수에 따라 바뀜
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
# BERTClassifier 인스턴스 생성
# - bertmodel: 미리 학습된 BERT 모델
# - dr_rate: 드롭아웃 비율 (옵션, 0.5로 설정)
# 드롭 아웃 -> 드롭아웃은 모델이 학습할 때 무작위로 일부 뉴런을 비활성화하여
#              모델의 일반화 성능을 향상시키는 데 사용.
#              여기서는 0.5로 설정되어 있어 각 뉴런이 50%의 확률로 비활성화.
# .to(device): 모델을 GPU에서 사용

model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [ ]:
# Optimizer와 학습 스케줄러 설정 (linear warmup and decay)
# 모델의 일반화를 향상시키고 오버피팅을 방지하는 데 도움

# 가중치 감쇠를 적용하지 않을 파라미터 그룹을 정의
no_decay = ['bias', 'LayerNorm.weight']

# 가중치 감쇠를 적용하지 않는 파라미터 그룹은 weight_decay를 0.0으로 설정하고,
# 적용하는 파라미터 그룹은 weight_decay를 0.01로 설정
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [ ]:
# Optimizer 설정: AdamW를 사용하며, 설정된 파라미터 그룹과 학습률을 적용
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)

# 손실 함수 설정: CrossEntropyLoss를 사용
loss_fn = nn.CrossEntropyLoss()

# 전체 학습 스텝 수 및 웜업 스텝 수 계산
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

# 스케줄러 설정
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
# 정확도 계산 함수
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)

    # 정확도 계산: 예측값과 실제 레이블이 일치하는 경우를 계산
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0

    # 모델을 학습 모드로 설정
    model.train()

    # 모델 학습
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)

        # 옵티마이저 업데이트 및 학습률 스케줄러 업데이트
        optimizer.step()
        scheduler.step()

        # 정확도 누적
        train_acc += calc_accuracy(out, label)

        # 일정 간격으로 학습 결과 출력
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))

    # 전체 데이터에 대해 한번의 학습이 끝난 후(에폭이 끝난 후) 학습 정확도 출력
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))

    # 모델을 평가 모드로 설정
    model.eval()

    # 테스트 데이터에 대한 정확도 계산
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)

        # 모델의 정확도 계산
        test_acc += calc_accuracy(out, label)

    # 에폭 종료 후 테스트 정확도 출력
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

<ipython-input-111-c711125859bb>:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/729 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 1.8098689317703247 train acc 0.234375
epoch 1 batch id 201 loss 0.9685375094413757 train acc 0.47007151741293535
epoch 1 batch id 401 loss 0.9573136568069458 train acc 0.543446072319202
epoch 1 batch id 601 loss 1.0134074687957764 train acc 0.5738092762063228
epoch 1 train acc 0.5857410265203474


<ipython-input-111-c711125859bb>:38: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):


  0%|          | 0/183 [00:00<?, ?it/s]

epoch 1 test acc 0.6384538446526151


In [ ]:
model1 = model # 원본 모델 복사

In [ ]:
# 모델을 통한 예측
def predict(predict_sentence):

  # 입력 문장과 임의의 레이블 데이터 생성
  data = [predict_sentence, '0']
  dataset_another = [data]

  # 입력 문장도 똑같이 토큰화
  another_test = BERTDataset(dataset_another, 0, 1, tokenizer, vocab, max_len, True, False)
  test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)

  # 모델을 평가 모드로 설정
  model1.eval()

  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
          token_ids = token_ids.long().to(device)
          segment_ids = segment_ids.long().to(device)
          valid_length= valid_length
          label = label.long().to(device)

          # 모델을 통해 예측 수행
          out = model1(token_ids, valid_length, segment_ids)

          for i in out:

            # 소프트맥스 함수를 적용하여 확률값 계산
            probs = torch.nn.functional.softmax(out, dim=-1)
            probs = probs.detach().cpu().numpy()

            # 가장 높은 확률을 갖는 클래스 선택
            predicted_class = np.argmax(probs, axis=1)[0]

            # 확률 계산
            probability = probs[0, predicted_class]*100

            # 결과에 따라 출력할 감정 카테고리
            class_names = ["불안", "당황", "분노", "슬픔", "평온", "행복"]

            # 결과 및 확률 출력
            result = f">> 입력하신 내용은 {probability:.2f}% 확률로 {class_names[predicted_class]}으로 예측됩니다."
            print(result)


In [ ]:
# 질문 무한 반복, 1을 입력하면 종료
while True:
  sentence = input("하고싶은 말을 입력해주세요 : ")
  if sentence == "1":
    break
  print(sentence)
  predict(sentence)
  print("\n")

하고싶은 말을 입력해주세요 : 집 가고 싶어
집 가고 싶어


<ipython-input-113-3649a4277b24>:15: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):


  0%|          | 0/1 [00:00<?, ?it/s]

>> 입력하신 내용은 32.07% 확률로 슬픔으로 예측됩니다.


하고싶은 말을 입력해주세요 : 나 너무 화나
나 너무 화나


  0%|          | 0/1 [00:00<?, ?it/s]

...

>> 입력하신 내용은 40.37% 확률로 분노으로 예측됩니다.


하고싶은 말을 입력해주세요 : 오늘은 아무일도 없어
오늘은 아무일도 없어


  0%|          | 0/1 [00:00<?, ?it/s]

>> 입력하신 내용은 55.99% 확률로 슬픔으로 예측됩니다.


하고싶은 말을 입력해주세요 : 심심해
심심해


  0%|          | 0/1 [00:00<?, ?it/s]

>> 입력하신 내용은 37.95% 확률로 불안으로 예측됩니다.


하고싶은 말을 입력해주세요 : 1


#===============================================

#음성데이터셋 KoBERT 모델 시도 결과

In [ ]:
import pandas as pd
from collections import Counter

# CSV 파일 불러오기
df1 = pd.read_csv('/content/4차년도.csv', encoding='cp949')
df2 = pd.read_csv('/content/5차년도.csv', encoding='cp949')
df3 = pd.read_csv('/content/5차년도_2차.csv', encoding='cp949')


In [ ]:
# 필요없는 열 삭제
df1 = df1[['발화문', '상황', '1번 감정', '2번 감정', '3번 감정', '4번 감정', '5번 감정']]
df2 = df2[['발화문', '상황', '1번 감정', '2번 감정', '3번 감정', '4번 감정', '5번 감정']]
df3 = df3[['발화문', '상황', '1번 감정', '2번 감정', '3번 감정', '4번 감정', '5번 감정']]

# 가장 많이 나온 감정을 찾는 함수
def most_frequent_emotion(row):
    emotions = [row['1번 감정'], row['2번 감정'], row['3번 감정'], row['4번 감정'], row['5번 감정']]
    emotion_counts = pd.Series(emotions).value_counts()

    # 감정 카운트 시리즈의 길이 확인
    if len(emotion_counts) == 0 or (len(emotion_counts) > 1 and emotion_counts.iloc[0] == emotion_counts.iloc[1]):
        # 동률인 경우 상황 반환
        return row['상황']
    else:
        # 가장 많은 감정 반환
        return emotion_counts.idxmax()

# 가장 많이 나온 감정을 새 컬럼에 추가
df1['가장_많이_나온_감정'] = df1.apply(most_frequent_emotion, axis=1)
df2['가장_많이_나온_감정'] = df2.apply(most_frequent_emotion, axis=1)
df3['가장_많이_나온_감정'] = df3.apply(most_frequent_emotion, axis=1)

# 필요없는 열 삭제
df1 = df1[['발화문', '가장_많이_나온_감정']]
df2 = df2[['발화문', '가장_많이_나온_감정']]
df3 = df3[['발화문', '가장_많이_나온_감정']]


# 열 이름 변경 (Sentence와 Emotion으로 통일)
df1.rename(columns={'발화문': 'Sentence', '가장_많이_나온_감정': 'Emotion'}, inplace=True)
df2.rename(columns={'발화문': 'Sentence', '가장_많이_나온_감정': 'Emotion'}, inplace=True)
df3.rename(columns={'발화문': 'Sentence', '가장_많이_나온_감정': 'Emotion'}, inplace=True)

# 데이터프레임 합치기
combined_df = pd.concat([df1, df2, df3], ignore_index=True)

# Emotion열의 모든 값을 소문자로 변경
combined_df['Emotion'] = combined_df['Emotion'].str.lower()

# Sentence열을 기준으로 중복된 행 제거
combined_df = combined_df.drop_duplicates(subset='Sentence', keep='first')

# 인덱스를 0부터 시작하는 새로운 인덱스로 재설정
combined_df = combined_df.reset_index(drop=True)
# 'Emotion' 열에서 'disgust'를 '분노'로 변경
combined_df['Emotion'] = combined_df['Emotion'].replace('disgust', '분노')

# 'Emotion' 열에서 'angry'를 '분노'로 변경
combined_df['Emotion'] = combined_df['Emotion'].replace('angry', '분노')

# 'Emotion' 열에서 'natural'를 '평온'로 변경
combined_df['Emotion'] = combined_df['Emotion'].replace('neutral', '평온')

# 'Emotion' 열에서 'happiness'를 '기쁨'로 변경
combined_df['Emotion'] = combined_df['Emotion'].replace('happiness', '기쁨')

# 'Emotion' 열에서 'surprise'를 '당황'로 변경
combined_df['Emotion'] = combined_df['Emotion'].replace('surprise', '당황')

# 'Emotion' 열에서 'sadness'를 '슬픔'로 변경
combined_df['Emotion'] = combined_df['Emotion'].replace('sadness', '슬픔')

# 'Emotion' 열에서 'fear'를 '불안'로 변경
combined_df['Emotion'] = combined_df['Emotion'].replace('fear', '불안')

# 'Emotion' 열에서 'sad'를 '슬픔'으로 변경
combined_df['Emotion'] = combined_df['Emotion'].replace('sad', '슬픔')

# 'Emotion' 열에서 'anger'를 '분노'로 변경
combined_df['Emotion'] = combined_df['Emotion'].replace('anger', '분노')
# 'Emotion' 열의 고유한 값들을 찾기
unique_emotions = combined_df['Emotion'].unique()

<ipython-input-2-47df02852bc1>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1.rename(columns={'발화문': 'Sentence', '가장_많이_나온_감정': 'Emotion'}, inplace=True)
<ipython-input-2-47df02852bc1>:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2.rename(columns={'발화문': 'Sentence', '가장_많이_나온_감정': 'Emotion'}, inplace=True)
<ipython-input-2-47df02852bc1>:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3.rename(columns={'발화문': 'Sentence

In [ ]:
# 감정 카테고리 숫자 데이터로 변환

combined_df.loc[(combined_df['Emotion'] == "불안"), 'Emotion'] = 0  #불안 => 0
combined_df.loc[(combined_df['Emotion'] == "당황"), 'Emotion'] = 1  #당 => 1
combined_df.loc[(combined_df['Emotion'] == "분노"), 'Emotion'] = 2  #분노 => 2
combined_df.loc[(combined_df['Emotion'] == "슬픔"), 'Emotion'] = 3  #슬픔 => 3
combined_df.loc[(combined_df['Emotion'] == "평온"), 'Emotion'] = 4  #평온 => 4
combined_df.loc[(combined_df['Emotion'] == "기쁨"), 'Emotion'] = 5  #행복 => 5

In [ ]:
# 학습용, 테스트용 데이터분리
from sklearn.model_selection import train_test_split

X = combined_df[['Sentence']]
y = combined_df['Emotion']

# 데이터 분리, 20%의 데이터를 테스트용으로 사용
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 나눠진 데이터 개수 확인
print("학습 데이터 개수:", len(X_train))
print("테스트 데이터 개수:", len(X_test))

학습 데이터 개수: 29769
테스트 데이터 개수: 7443


In [ ]:
# 학습용, 테스트용 데이터 생성
dataset_train = pd.concat([X_train, y_train], axis=1)
dataset_test = pd.concat([X_test, y_test], axis=1)

In [ ]:
from gluonnlp.data import TSVDataset

# tsv 데이터로 변환
# sep='\t'탭을 구분자로 사용하여 csv가 아닌 tsv파일로 변환한다
dataset_train.to_csv('dataset_train.tsv', sep='\t', index=False)
dataset_test.to_csv('dataset_test.tsv', sep='\t', index=False)

In [ ]:
# TSV 파일에서 데이터셋을 로드
# num_discard_samples=1 첫번째 행을 무시 -> 열 이름

dataset_train = nlp.data.TSVDataset('dataset_train.tsv', field_indices=[0, 1], num_discard_samples=1)
dataset_test = nlp.data.TSVDataset('dataset_test.tsv', field_indices=[0, 1], num_discard_samples=1)

In [ ]:
# 파라미터 세팅
max_len = 64            # 문장 최대 길이
batch_size = 64         # 한 번의 모델 업데이트를 위해 사용되는 데이터 샘플의 개수
warmup_ratio = 0.1      # 학습 초기에 적용되는 웜업 비율
num_epochs = 1          # 전체학습 반복 횟수
max_grad_norm = 1       # 그래디언트 폭주를 방지
log_interval = 200      # 학습 중 로그를 출력하는 간격
learning_rate =  5e-5   # 학습률

In [ ]:
# BERTDataset 클래스 내의 __init__ 메서드에서 레이블을 정수로 변환하도록 수정
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len,
                 pad, pair):
        transform = BERTSentenceTransform(bert_tokenizer, max_seq_length=max_len, vocab=vocab, pad=pad, pair=pair)
        self.sentences = [transform([i[sent_idx]]) for i in dataset]

        # 레이블을 숫자로 변환 (문자열이면 그대로 사용)
        self.labels = [int(i[label_idx]) if str(i[label_idx]).isdigit() else i[label_idx] for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return len(self.labels)


# KoBERT 토크나이저 및 BERT 모델 불러오기
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)

# BERTVocab을 사용하여 vocab 생성
# (vocab => 모델이 텍스트 데이터를 처리하는 과정에서 사용되는 어휘 집합)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')

# 학습 데이터셋과 테스트 데이터셋을 BERTDataset으로 변환
data_train = BERTDataset(dataset_train, 0, 1, tokenizer, vocab, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tokenizer, vocab, max_len, True, False)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [ ]:
# 데이터셋을 미니배치로 나누어 준다, 현재 설정된 batch_size = 64
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
# 감정 분류를 위한 분류 모델을 정의하는 클래스
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=6, # 클래스 수, 감정 분류 개수에 따라 바뀜
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
# BERTClassifier 인스턴스 생성
# - bertmodel: 미리 학습된 BERT 모델
# - dr_rate: 드롭아웃 비율 (옵션, 0.5로 설정)
# 드롭 아웃 -> 드롭아웃은 모델이 학습할 때 무작위로 일부 뉴런을 비활성화하여
#              모델의 일반화 성능을 향상시키는 데 사용.
#              여기서는 0.5로 설정되어 있어 각 뉴런이 50%의 확률로 비활성화.
# .to(device): 모델을 GPU에서 사용

model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [ ]:
# Optimizer와 학습 스케줄러 설정 (linear warmup and decay)
# 모델의 일반화를 향상시키고 오버피팅을 방지하는 데 도움

# 가중치 감쇠를 적용하지 않을 파라미터 그룹을 정의
no_decay = ['bias', 'LayerNorm.weight']

# 가중치 감쇠를 적용하지 않는 파라미터 그룹은 weight_decay를 0.0으로 설정하고,
# 적용하는 파라미터 그룹은 weight_decay를 0.01로 설정
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [ ]:
# Optimizer 설정: AdamW를 사용하며, 설정된 파라미터 그룹과 학습률을 적용
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)

# 손실 함수 설정: CrossEntropyLoss를 사용
loss_fn = nn.CrossEntropyLoss()

# 전체 학습 스텝 수 및 웜업 스텝 수 계산
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

# 스케줄러 설정
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
# 정확도 계산 함수
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)

    # 정확도 계산: 예측값과 실제 레이블이 일치하는 경우를 계산
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0

    # 모델을 학습 모드로 설정
    model.train()

    # 모델 학습
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length = valid_length.to(device)
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)

        # 옵티마이저 업데이트 및 학습률 스케줄러 업데이트
        optimizer.step()
        scheduler.step()

        # 정확도 누적
        train_acc += calc_accuracy(out, label)

        # 일정 간격으로 학습 결과 출력
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))

    # 전체 데이터에 대해 한번의 학습이 끝난 후(에폭이 끝난 후) 학습 정확도 출력
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))

    # 모델을 평가 모드로 설정
    model.eval()

    # 테스트 데이터에 대한 정확도 계산
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length = valid_length.to(device)
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)

        # 모델의 정확도 계산
        test_acc += calc_accuracy(out, label)

    # 에폭 종료 후 테스트 정확도 출력
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

<ipython-input-35-63a06f8c8a5c>:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/466 [00:00<?, ?it/s]

TypeError: ignored

In [ ]:
model1 = model # 원본 모델 복사

In [ ]:
# 모델을 통한 예측
def predict(predict_sentence):

  # 입력 문장과 임의의 레이블 데이터 생성
  data = [predict_sentence, '0']
  dataset_another = [data]

  # 입력 문장도 똑같이 토큰화
  another_test = BERTDataset(dataset_another, 0, 1, tokenizer, vocab, max_len, True, False)
  test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)

  # 모델을 평가 모드로 설정
  model1.eval()

  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
          token_ids = token_ids.long().to(device)
          segment_ids = segment_ids.long().to(device)
          valid_length= valid_length
          label = label.long().to(device)

          # 모델을 통해 예측 수행
          out = model1(token_ids, valid_length, segment_ids)

          for i in out:

            # 소프트맥스 함수를 적용하여 확률값 계산
            probs = torch.nn.functional.softmax(out, dim=-1)
            probs = probs.detach().cpu().numpy()

            # 가장 높은 확률을 갖는 클래스 선택
            predicted_class = np.argmax(probs, axis=1)[0]

            # 확률 계산
            probability = probs[0, predicted_class]*100

            # 결과에 따라 출력할 감정 카테고리
            class_names = ["불안", "당황", "분노", "슬픔", "평온", "행복"]

            # 결과 및 확률 출력
            result = f">> 입력하신 내용은 {probability:.2f}% 확률로 {class_names[predicted_class]}으로 예측됩니다."
            print(result)


In [ ]:
# 질문 무한 반복, 1을 입력하면 종료
while True:
  sentence = input("하고싶은 말을 입력해주세요 : ")
  if sentence == "1":
    break
  print(sentence)
  predict(sentence)
  print("\n")

#===============================================

#3. 병합 데이터셋 KoBERT 모델 시도 결과

In [ ]:
import pandas as pd

# 엑셀 파일 불러오기
df1 = pd.read_excel('/content/감성대화말뭉치(최종데이터)_Training.xlsx')
df2 = pd.read_excel('/content/감성대화말뭉치(최종데이터)_Validation.xlsx')
df3 = pd.read_excel('/content/한국어_단발성_대화_데이터셋.xlsx')

In [ ]:
# 필요없는 열 삭제 (문장과 감정만 남김)
df1 = df1[['사람문장1', '감정_대분류']]
df2 = df2[['사람문장1', '감정_대분류']]
df3 = df3[['Sentence', 'Emotion']]

# 열 이름 변경 (Sentence와 Emotion으로 통일)
df1.rename(columns={'사람문장1': 'Sentence', '감정_대분류': 'Emotion'}, inplace=True)
df2.rename(columns={'사람문장1': 'Sentence', '감정_대분류': 'Emotion'}, inplace=True)

# 데이터프레임 합치기 (감성대화말뭉치)
combined_df = pd.concat([df1, df2], ignore_index=True)

# 'Emotion' 열에서 '상처'를 '슬픔'으로 변경 (감성대화말뭉치)
combined_df['Emotion'] = combined_df['Emotion'].replace('상처', '슬픔')

# 'Emotion' 열에서 '혐오'를 '분노'로 변경 (한국어_단발성_대화_데이터셋)
df3['Emotion'] = df3['Emotion'].replace('혐오', '분노')

# 'Emotion' 열에서 '중립'을 '평온'으로 변경 (한국어_단발성_대화_데이터셋)
df3['Emotion'] = df3['Emotion'].replace('중립', '평온')

# 'Emotion' 열에서 '공포'를 '불안'으로 변경 (한국어_단발성_대화_데이터셋)
df3['Emotion'] = df3['Emotion'].replace('공포', '불안')

# 'Emotion' 열에서 '놀람'을 '당황'으로 변경 (한국어_단발성_대화_데이터셋)
df3['Emotion'] = df3['Emotion'].replace('놀람', '당황')

# 'Emotion' 열에서 '행복'을 '기쁨'으로 변경 (한국어_단발성_대화_데이터셋)
df3['Emotion'] = df3['Emotion'].replace('행복', '기쁨')

# 데이터프레임 합치기 (한국어_단발성_대화_데이터셋 + 감성대화말뭉치)
final_df = pd.concat([combined_df, df3], ignore_index=True)

# 줄바꿈 문자를 공백으로 대체 ('TSVDataset' 로드할 때 오류가 발생해서)
final_df['Sentence'] = final_df['Sentence'].str.replace('\n', ' ', regex=True)

<ipython-input-119-a23e459f091b>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1.rename(columns={'사람문장1': 'Sentence', '감정_대분류': 'Emotion'}, inplace=True)
<ipython-input-119-a23e459f091b>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2.rename(columns={'사람문장1': 'Sentence', '감정_대분류': 'Emotion'}, inplace=True)
<ipython-input-119-a23e459f091b>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vi

In [ ]:
# 감정 카테고리 숫자 데이터로 변환

final_df.loc[(final_df['Emotion'] == "불안"), 'Emotion'] = 0  #공포 => 0
final_df.loc[(final_df['Emotion'] == "당황"), 'Emotion'] = 1  #놀람 => 1
final_df.loc[(final_df['Emotion'] == "분노"), 'Emotion'] = 2  #분노 => 2
final_df.loc[(final_df['Emotion'] == "슬픔"), 'Emotion'] = 3  #슬픔 => 3
final_df.loc[(final_df['Emotion'] == "평온"), 'Emotion'] = 4  #평온 => 4
final_df.loc[(final_df['Emotion'] == "기쁨"), 'Emotion'] = 5  #행복 => 5

In [ ]:
# 학습용, 테스트용 데이터분리
from sklearn.model_selection import train_test_split

X = final_df[['Sentence']]
y = final_df['Emotion']

# 데이터 분리, 20%의 데이터를 테스트용으로 사용
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 나눠진 데이터 개수 확인
print("학습 데이터 개수:", len(X_train))
print("테스트 데이터 개수:", len(X_test))

학습 데이터 개수: 77492
테스트 데이터 개수: 19373


In [ ]:
# 학습용, 테스트용 데이터 생성
dataset_train = pd.concat([X_train, y_train], axis=1)
dataset_test = pd.concat([X_test, y_test], axis=1)

In [ ]:
from gluonnlp.data import TSVDataset

# tsv 데이터로 변환
# sep='\t'탭을 구분자로 사용하여 csv가 아닌 tsv파일로 변환한다
dataset_train.to_csv('dataset_train.tsv', sep='\t', index=False)
dataset_test.to_csv('dataset_test.tsv', sep='\t', index=False)

In [ ]:
# TSV 파일에서 데이터셋을 로드
# num_discard_samples=1 첫번째 행을 무시 -> 열 이름

dataset_train = nlp.data.TSVDataset('dataset_train.tsv', field_indices=[0, 1], num_discard_samples=1)
dataset_test = nlp.data.TSVDataset('dataset_test.tsv', field_indices=[0, 1], num_discard_samples=1)

In [ ]:
# 파라미터 세팅
max_len = 64            # 문장 최대 길이
batch_size = 64         # 한 번의 모델 업데이트를 위해 사용되는 데이터 샘플의 개수
warmup_ratio = 0.1      # 학습 초기에 적용되는 웜업 비율
num_epochs = 1          # 전체학습 반복 횟수
max_grad_norm = 1       # 그래디언트 폭주를 방지
log_interval = 200      # 학습 중 로그를 출력하는 간격
learning_rate =  5e-5   # 학습률

In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len,
                 pad, pair):

        # BERTSentenceTransform을 사용하여 데이터셋의 문장과 레이블을 변환
        transform = BERTSentenceTransform(bert_tokenizer, max_seq_length=max_len,vocab=vocab, pad=pad, pair=pair)
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        # 변환된 문장과 레이블을 반환
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

# KoBERT 토크나이저 및 BERT 모델 불러오기
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)

# BERTVocab을 사용하여 vocab 생성
# (vocab => 모델이 텍스트 데이터를 처리하는 과정에서 사용되는 어휘 집합)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')

# 학습 데이터셋과 테스트 데이터셋을 BERTDataset으로 변환
data_train = BERTDataset(dataset_train, 0, 1, tokenizer, vocab, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tokenizer, vocab, max_len, True, False)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [ ]:
# 데이터셋을 미니배치로 나누어 준다, 현재 설정된 batch_size = 64
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
# 감정 분류를 위한 분류 모델을 정의하는 클래스
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=6, # 클래스 수, 감정 분류 개수에 따라 바뀜
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
# BERTClassifier 인스턴스 생성
# - bertmodel: 미리 학습된 BERT 모델
# - dr_rate: 드롭아웃 비율 (옵션, 0.5로 설정)
# 드롭 아웃 -> 드롭아웃은 모델이 학습할 때 무작위로 일부 뉴런을 비활성화하여
#              모델의 일반화 성능을 향상시키는 데 사용.
#              여기서는 0.5로 설정되어 있어 각 뉴런이 50%의 확률로 비활성화.
# .to(device): 모델을 GPU에서 사용

model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [ ]:
# Optimizer와 학습 스케줄러 설정 (linear warmup and decay)
# 모델의 일반화를 향상시키고 오버피팅을 방지하는 데 도움

# 가중치 감쇠를 적용하지 않을 파라미터 그룹을 정의
no_decay = ['bias', 'LayerNorm.weight']

# 가중치 감쇠를 적용하지 않는 파라미터 그룹은 weight_decay를 0.0으로 설정하고,
# 적용하는 파라미터 그룹은 weight_decay를 0.01로 설정
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [ ]:
# Optimizer 설정: AdamW를 사용하며, 설정된 파라미터 그룹과 학습률을 적용
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)

# 손실 함수 설정: CrossEntropyLoss를 사용
loss_fn = nn.CrossEntropyLoss()

# 전체 학습 스텝 수 및 웜업 스텝 수 계산
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

# 스케줄러 설정
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
# 정확도 계산 함수
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)

    # 정확도 계산: 예측값과 실제 레이블이 일치하는 경우를 계산
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0

    # 모델을 학습 모드로 설정
    model.train()

    # 모델 학습
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)

        # 옵티마이저 업데이트 및 학습률 스케줄러 업데이트
        optimizer.step()
        scheduler.step()

        # 정확도 누적
        train_acc += calc_accuracy(out, label)

        # 일정 간격으로 학습 결과 출력
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))

    # 전체 데이터에 대해 한번의 학습이 끝난 후(에폭이 끝난 후) 학습 정확도 출력
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))

    # 모델을 평가 모드로 설정
    model.eval()

    # 테스트 데이터에 대한 정확도 계산
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)

        # 모델의 정확도 계산
        test_acc += calc_accuracy(out, label)

    # 에폭 종료 후 테스트 정확도 출력
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

<ipython-input-134-c711125859bb>:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/1211 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 1.8682494163513184 train acc 0.15625
epoch 1 batch id 201 loss 1.1034774780273438 train acc 0.3853389303482587
epoch 1 batch id 401 loss 1.2598042488098145 train acc 0.4791926433915212
epoch 1 batch id 601 loss 1.1366702318191528 train acc 0.5173668885191348
epoch 1 batch id 801 loss 0.8883556723594666 train acc 0.5419397627965044
epoch 1 batch id 1001 loss 0.9527384638786316 train acc 0.558425949050949
epoch 1 batch id 1201 loss 0.9873826503753662 train acc 0.5711906744379683
epoch 1 train acc 0.5720151416502572


<ipython-input-134-c711125859bb>:38: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):


  0%|          | 0/303 [00:00<?, ?it/s]

epoch 1 test acc 0.6364468738540521


In [ ]:
model1 = model # 원본 모델 복사

In [ ]:
# 모델을 통한 예측
def predict(predict_sentence):

  # 입력 문장과 임의의 레이블 데이터 생성
  data = [predict_sentence, '0']
  dataset_another = [data]

  # 입력 문장도 똑같이 토큰화
  another_test = BERTDataset(dataset_another, 0, 1, tokenizer, vocab, max_len, True, False)
  test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)

  # 모델을 평가 모드로 설정
  model1.eval()

  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
          token_ids = token_ids.long().to(device)
          segment_ids = segment_ids.long().to(device)
          valid_length= valid_length
          label = label.long().to(device)

          # 모델을 통해 예측 수행
          out = model1(token_ids, valid_length, segment_ids)

          for i in out:

            # 소프트맥스 함수를 적용하여 확률값 계산
            probs = torch.nn.functional.softmax(out, dim=-1)
            probs = probs.detach().cpu().numpy()

            # 가장 높은 확률을 갖는 클래스 선택
            predicted_class = np.argmax(probs, axis=1)[0]

            # 확률 계산
            probability = probs[0, predicted_class]*100

            # 결과에 따라 출력할 감정 카테고리
            class_names = ["불안", "당황", "분노", "슬픔", "평온", "행복",]

            # 결과 및 확률 출력
            result = f">> 입력하신 내용은 {probability:.2f}% 확률로 {class_names[predicted_class]}으로 예측됩니다."
            print(result)


In [ ]:
# 질문 무한 반복, 1을 입력하면 종료
while True:
  sentence = input("하고싶은 말을 입력해주세요 : ")
  if sentence == "1":
    break
  print(sentence)
  predict(sentence)
  print("\n")

하고싶은 말을 입력해주세요 : 집 가고 싶어
집 가고 싶어


<ipython-input-136-6f25d4f51667>:15: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):


  0%|          | 0/1 [00:00<?, ?it/s]

>> 입력하신 내용은 49.30% 확률로 슬픔으로 예측됩니다.


하고싶은 말을 입력해주세요 : 나 너무 화나
나 너무 화나


  0%|          | 0/1 [00:00<?, ?it/s]

...

>> 입력하신 내용은 79.95% 확률로 분노으로 예측됩니다.


하고싶은 말을 입력해주세요 : 오늘은 아무일도 없어
오늘은 아무일도 없어


  0%|          | 0/1 [00:00<?, ?it/s]

>> 입력하신 내용은 59.73% 확률로 슬픔으로 예측됩니다.


하고싶은 말을 입력해주세요 : 심심해
심심해


  0%|          | 0/1 [00:00<?, ?it/s]

>> 입력하신 내용은 37.83% 확률로 불안으로 예측됩니다.


하고싶은 말을 입력해주세요 : 1


#===============================================

#4. 병합 데이터셋 다른 모델 시도 결과


In [ ]:
import pandas as pd

# 엑셀 파일 불러오기
df1 = pd.read_excel('/content/감성대화말뭉치(최종데이터)_Training.xlsx')
df2 = pd.read_excel('/content/감성대화말뭉치(최종데이터)_Validation.xlsx')
df3 = pd.read_excel('/content/한국어_단발성_대화_데이터셋.xlsx')

In [ ]:
# 필요없는 열 삭제 (문장과 감정만 남김)
df1 = df1[['사람문장1', '감정_대분류']]
df2 = df2[['사람문장1', '감정_대분류']]
df3 = df3[['Sentence', 'Emotion']]

# 열 이름 변경 (Sentence와 Emotion으로 통일)
df1.rename(columns={'사람문장1': 'Sentence', '감정_대분류': 'Emotion'}, inplace=True)
df2.rename(columns={'사람문장1': 'Sentence', '감정_대분류': 'Emotion'}, inplace=True)

# 데이터프레임 합치기 (감성대화말뭉치)
combined_df = pd.concat([df1, df2], ignore_index=True)

# 'Emotion' 열에서 '상처'를 '슬픔'으로 변경 (감성대화말뭉치)
combined_df['Emotion'] = combined_df['Emotion'].replace('상처', '슬픔')

# 변경된 데이터프레임 확인
combined_df[combined_df['Emotion'] == '슬픔']

# 'Emotion' 열에서 '혐오'를 '분노'로 변경 (한국어_단발성_대화_데이터셋)
df3['Emotion'] = df3['Emotion'].replace('혐오', '분노')

# 변경된 데이터프레임 확인
df3[df3['Emotion'] == '분노']

# 'Emotion' 열에서 '중립'을 '평온'으로 변경 (한국어_단발성_대화_데이터셋)
df3['Emotion'] = df3['Emotion'].replace('중립', '평온')

# 변경된 데이터프레임 확인
df3[df3['Emotion'] == '평온']

# 'Emotion' 열에서 '공포'를 '불안'으로 변경 (한국어_단발성_대화_데이터셋)
df3['Emotion'] = df3['Emotion'].replace('공포', '불안')

# 'Emotion' 열에서 '놀람'을 '당황'으로 변경 (한국어_단발성_대화_데이터셋)
df3['Emotion'] = df3['Emotion'].replace('놀람', '당황')

# 'Emotion' 열에서 '행복'을 '기쁨'으로 변경 (한국어_단발성_대화_데이터셋)
df3['Emotion'] = df3['Emotion'].replace('행복', '기쁨')
# 데이터프레임 합치기 (한국어_단발성_대화_데이터셋 + 감성대화말뭉치)
final_df = pd.concat([combined_df, df3], ignore_index=True)

In [ ]:
# 데이터를 훈련 및 테스트 세트로 분할
X = final_df['Sentence']  #: Sentence 열을 사용
y = final_df['Emotion']   #: Emotion 열을 사용
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

# 데이터를 훈련 및 테스트 세트로 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# TF-IDF 변환기 초기화
tfidf_vectorizer = TfidfVectorizer(max_features=100)  # 최대 특징 수를 조절할 수 있음

# 훈련 데이터를 TF-IDF로 변환
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)

# 테스트 데이터를 TF-IDF로 변환
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# 모델 학습
# 나이브 베이즈 모델 학습
nb_model = MultinomialNB()
nb_model.fit(X_train_tfidf, y_train)

# k-최근접 이웃 모델 학습
knn_model = KNeighborsClassifier()
knn_model.fit(X_train_tfidf, y_train)

# 의사결정 나무 모델 학습
dt_model = DecisionTreeClassifier()
dt_model.fit(X_train_tfidf, y_train)

# 앙상블 모델 ( 랜덤 포레스트) 학습
rf_model = RandomForestClassifier()
rf_model.fit(X_train_tfidf, y_train)


In [ ]:
from sklearn.metrics import accuracy_score, classification_report

# 각 모델의 예측
nb_pred = nb_model.predict(X_test_tfidf)
knn_pred = knn_model.predict(X_test_tfidf)
dt_pred = dt_model.predict(X_test_tfidf)
rf_pred = rf_model.predict(X_test_tfidf)

# 정확도 출력
print("Naive Bayes Accuracy:", accuracy_score(y_test, nb_pred))
print("K-Nearest Neighbors Accuracy:", accuracy_score(y_test, knn_pred))
print("Decision Tree Accuracy:", accuracy_score(y_test, dt_pred))
print("Random Forest Accuracy:", accuracy_score(y_test, rf_pred))

# Classification Report 출력
print("Naive Bayes Report:\n", classification_report(y_test, nb_pred))
print("K-Nearest Neighbors Report:\n", classification_report(y_test, knn_pred))
print("Decision Tree Report:\n", classification_report(y_test, dt_pred))
print("Random Forest Report:\n", classification_report(y_test, rf_pred))